# LGBM Model

The winning solution uses combination of four models and lgbm is one of the model. I will start by looking at this solution. 

[Winning Solution](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/47582#latest-605356)


In [1]:
"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding more
average features and weekly average features on it.
"""
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

In [2]:
data_dir = 'Location'

In [3]:
class readDataStore():
    """
    This module is to read the data from data store.

    In our case, our date store is local and is in .csv format
    """

    def __init__(self, data_dir):
        self.data_dir = data_dir

    def read_stores(self, file_name):
        df = pd.read_csv(self.data_dir + file_name)

        return df

    def read_items(self, file_name):
        df = pd.read_csv(self.data_dir + file_name)

        return df

    def read_transactions(self, file_name):
        df = pd.read_csv(self.data_dir + file_name)

        return df

    def read_oil(self, file_name):
        df = pd.read_csv(self.data_dir + file_name)

        return df

    def read_holidays(self, file_name):
        df = pd.read_csv(self.data_dir + file_name)

        return df

    def read_train(self, file_name):
        df = pd.read_csv(self.data_dir + file_name)

        return df

    def read_test(self, file_name):
        df = pd.read_csv(self.data_dir + file_name, usecols=[0, 1, 2, 3, 4],
                         dtype={'onpromotion': bool},
                         parse_dates=["date"]  # , date_parser=parser
                         ).set_index(
            ['store_nbr', 'item_nbr', 'date']
        )

        return df

    def read_train(self, file_name):
        df = pd.read_csv(self.data_dir + file_name,
                         usecols=[1, 2, 3, 4, 5],
                         dtype={'onpromotion': bool},
                         converters={'unit_sales': lambda u: np.log1p(
                             float(u)) if float(u) > 0 else 0},
                         parse_dates=["date"],
                         skiprows=range(1, 66458909)  # 2016-01-01
                         )

        return df

In [4]:
rd = readDataStore(data_dir)
df_test = rd.read_test('test.csv')

# Read the items data frame
df_items = rd.read_items('items.csv').set_index("item_nbr")

# Read the stores data frame
df_stores = rd.read_items('stores.csv').set_index("store_nbr")

# Reading the training data
df_train = rd.read_train('train.csv')


In [6]:
# Assiging the values to the labels
le = LabelEncoder()
df_items['family'] = le.fit_transform(df_items['family'].values)

df_stores['city'] = le.fit_transform(df_stores['city'].values)
df_stores['state'] = le.fit_transform(df_stores['state'].values)
df_stores['type'] = le.fit_transform(df_stores['type'].values)

items = df_items.copy()
stores = df_stores.copy()

In [7]:
# Extracting the training data for 2017 until 2017-08-14
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
# Delete the big training data
# del df_train
# On each store what items are on promotion on each day
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

#Training data fro 2017 consists of 23 billions records an dit covers only 8.5 months. 
# This problem becomes complex and computational expensive if we have to use more data andmore derived features.

# promotion columns from training data
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

# promotion data from test data
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)

# Test data Strats from 2017-08-16 to 2017-08-31

# promo_2017_test.sum(axis=1).plot()

promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

# Combined the promotion data from training and test data
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

## Working on Traget Variable: 

# In this problem, our traget variable is unit_sales.

df_2017_tmp = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1)

# df_2017_tmp.count(axis=0).plot()

There are some missing values for unit sales ascan be seen from above plot. In the solution, they filled with 0, however I am not fully convinced. There could be other method to handle these missing values. One of the argument is that missing values are due to issues with data colllection. 

This issue can be further studied. 

On epotentional candidate to be used for forward filling can be average over last 20 days or so. 

In [8]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)

df_2017.columns = df_2017.columns.get_level_values(1)


items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

# df_2017_item.sum(axis=0).plot()
# promo_2017_item.sum(axis=0).plot()

df_2017_store_class = df_2017.reset_index()
df_2017_store_class['class'] = items['class'].values
df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

# df_2017_store_class.sum(axis=1).plot()

df_2017_promo_store_class = promo_2017.reset_index()
df_2017_promo_store_class['class'] = items['class'].values
df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

def get_timespan(df, dt, minus, periods, freq='D'):
    
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]


def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "promo_14_2017": get_timespan(df = promo_df, dt= t2017, minus=14, periods=14, freq='D').sum(axis=1).values,
        "promo_60_2017": get_timespan(df = promo_df, dt = t2017, minus = 60, periods = 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(df = promo_df, dt = t2017, minus = 140, periods = 140).sum(axis=1).values,
        "promo_3_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 3).sum(axis=1).values,
        "promo_7_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 7).sum(axis=1).values,
        "promo_14_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 14).sum(axis=1).values,
    }

    for i in [3, 7, 14, 30, 60, 140]:
        tmp1 = get_timespan(df, t2017, i, i)
        tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1

        X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
        X['has_promo_mean_%s_decay' % i] = (tmp1 * tmp2.replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

        X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values
        X['no_promo_mean_%s_decay' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017 + timedelta(days=-7), i, i)
        X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s_2' % i] = tmp.mean(axis=1).values
        X['median_%s_2' % i] = tmp.median(axis=1).values
        X['min_%s_2' % i] = tmp.min(axis=1).values
        X['max_%s_2' % i] = tmp.max(axis=1).values
        X['std_%s_2' % i] = tmp.std(axis=1).values

    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

        tmp = get_timespan(promo_df, t2017, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)
    X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values
    X['last_has_promo_day_in_after_15_days'] = i - ((tmp > 0) * np.arange(15)).max(axis=1).values
    X['first_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values

    for i in range(1, 16):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values

    for i in range(-16, 16):
        X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = df[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X

print("Preparing dataset...")
t2017 = date(2017, 6, 14)
num_days = 6
X_l, y_l = [], []
for i in range(num_days):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)

    X_tmp2 = prepare_dataset(df_2017_item, promo_2017_item, t2017 + delta, is_train=False, name_prefix='item')
    X_tmp2.index = df_2017_item.index
    X_tmp2 = X_tmp2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

    X_tmp3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, t2017 + delta, is_train=False, name_prefix='store_class')
    X_tmp3.index = df_2017_store_class.index
    X_tmp3 = X_tmp3.reindex(df_2017_store_class_index).reset_index(drop=True)

    X_tmp = pd.concat([X_tmp, X_tmp2, X_tmp3, items.reset_index(), stores.reset_index()], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

    del X_tmp2
    gc.collect()

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

del X_l, y_l

# Prepare Validation data set
val_start_date = date(2017, 7, 26)
X_val, y_val = prepare_dataset(df_2017, promo_2017, val_start_date )

X_val2 = prepare_dataset(df_2017_item, promo_2017_item, val_start_date, is_train=False, name_prefix='item')
X_val2.index = df_2017_item.index
X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_val3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, val_start_date, is_train=False, name_prefix='store_class')
X_val3.index = df_2017_store_class.index
X_val3 = X_val3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_val = pd.concat([X_val, X_val2, X_val3, items.reset_index(), stores.reset_index()], axis=1)

# Prepare Test data set
test_start_date = date(2017, 8, 16)
X_test = prepare_dataset(df_2017, promo_2017,test_start_date , is_train=False)

X_test2 = prepare_dataset(df_2017_item, promo_2017_item,test_start_date, is_train=False, name_prefix='item')
X_test2.index = df_2017_item.index
X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_test3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, test_start_date, is_train=False, name_prefix='store_class')
X_test3.index = df_2017_store_class.index
X_test3 = X_test3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_test = pd.concat([X_test, X_test2, X_test3, items.reset_index(), stores.reset_index()], axis=1)

del X_test2, X_val2, df_2017_item, promo_2017_item, df_2017_store_class, df_2017_promo_store_class, df_2017_store_class_index
gc.collect()

Preparing dataset...


0

# Training Model

In [9]:
OUTPUT_DIR = data_dir + 'output/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
print("Training and predicting models...")
params = {
    'num_leaves': 3,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 20
}

MAX_ROUNDS = 200
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

weight = items["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err))

y_val = np.array(val_pred).transpose()
df_preds = pd.DataFrame(
    y_val, index=df_2017.index,
    columns=pd.date_range("2017-07-26", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
df_preds["unit_sales"] = np.clip(np.expm1(df_preds["unit_sales"]), 0, 1000)
df_preds.reset_index().to_csv(OUTPUT_DIR + 'lgb_cv.csv', index=False)


print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)


submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv(OUTPUT_DIR + 'lgb_sub.csv', float_format='%.4f', index=None)

Training and predicting models...
Step 1


/home/researcher/Raza/pycharm_projects/tmp/retail-sales-prediction/venv/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.516811	valid_1's l2: 0.502299
[100]	training's l2: 0.384125	valid_1's l2: 0.375557
[150]	training's l2: 0.3429	valid_1's l2: 0.336831
[200]	training's l2: 0.3282	valid_1's l2: 0.323126
Did not meet early stopping. Best iteration is:
[200]	training's l2: 0.3282	valid_1's l2: 0.323126
mean_140_decay: 4024953.10
mean_60_decay: 2922402.25
mean_7_decay: 2619069.54
mean_30_decay: 1141487.40
mean_7: 752724.45
promo_0: 652552.72
mean_14_decay: 602075.80
mean_4_dow0_2017: 520040.69
mean_20_dow0_2017: 508434.13
median_7: 233157.57
day_1_2017: 158488.50
no_promo_mean_140_decay: 116449.29
no_promo_mean_7_decay: 79972.05
no_promo_mean_60_decay: 61348.96
mean_3_decay: 58255.24
last_has_sales_day_in_last_7: 49406.53
no_promo_mean_3_decay: 40563.23
no_promo_mean_3: 37003.33
no_promo_mean_7: 32569.45
min_3: 20231.08
first_has_promo_day_in_last_7: 12477.36
no_promo_mean_14_decay: 7195.39
mean_3: 6496.98
promo_-7: 4916.9

Step 2
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.497177	valid_1's l2: 0.500674
[100]	training's l2: 0.39025	valid_1's l2: 0.397242
[150]	training's l2: 0.35605	valid_1's l2: 0.363741
[200]	training's l2: 0.343303	valid_1's l2: 0.350865
Did not meet early stopping. Best iteration is:
[200]	training's l2: 0.343303	valid_1's l2: 0.350865
mean_140_decay: 5711426.42
mean_60_decay: 3222917.47
mean_7: 740104.04
mean_20_dow1_2017: 593289.06
mean_7_decay: 584144.63
median_7: 301114.51
first_has_promo_day_in_after_15_days: 250097.29
mean_30_decay: 185863.30
mean_4_dow1_2017: 128267.69
mean_14_decay: 127422.09
promo_1: 104272.61
no_promo_mean_7_decay: 62091.06
no_promo_mean_60: 61930.29
no_promo_mean_140_decay: 58259.06
no_promo_mean_60_decay: 33230.70
no_promo_mean_30: 23467.94
no_promo_mean_30_decay: 23088.49
no_promo_mean_3: 15714.33
no_promo_mean_7: 15132.44
max_7: 12619.80
no_promo_mean_14_decay: 11079.24
day_1_2017: 10423.20
no_promo_mean_14: 4289.

Step 3
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.542749	valid_1's l2: 0.560948
[100]	training's l2: 0.4122	valid_1's l2: 0.430593
[150]	training's l2: 0.371058	valid_1's l2: 0.388348
[200]	training's l2: 0.356517	valid_1's l2: 0.372993
Did not meet early stopping. Best iteration is:
[200]	training's l2: 0.356517	valid_1's l2: 0.372993
mean_140_decay: 5683389.58
mean_60_decay: 2664632.58
mean_4_dow2_2017: 1822269.24
mean_20_dow2_2017: 1448216.37
mean_7: 1299955.13
promo_2: 475208.97
median_7: 284090.63
day_5_2017: 102456.50
no_promo_mean_140_decay: 92599.97
no_promo_mean_7_decay: 78240.83
mean_7_decay: 57718.85
no_promo_mean_60_decay: 51731.68
max_7: 48675.04
mean_14_decay: 48060.34
mean_30_decay: 46934.10
day_4_2017: 19697.47
no_promo_mean_30_decay: 18315.44
no_promo_mean_14_decay: 9997.44
item_promo_2: 6375.34
no_promo_mean_3: 5786.35
no_promo_mean_14: 5747.34
no_promo_mean_7: 4219.06
no_promo_mean_3_decay: 3855.88
promo_14_2017: 0.00
promo_6

...

Step 13
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.582903	valid_1's l2: 0.573063
[100]	training's l2: 0.458736	valid_1's l2: 0.452824
[150]	training's l2: 0.418835	valid_1's l2: 0.415215
[200]	training's l2: 0.404232	valid_1's l2: 0.401335
Did not meet early stopping. Best iteration is:
[200]	training's l2: 0.404232	valid_1's l2: 0.401335
mean_140_decay: 6540969.67
mean_60_decay: 2680900.73
mean_20_dow5_2017: 857630.01
no_promo_mean_60: 805109.60
no_promo_mean_30: 694151.65
promo_12: 483367.88
mean_60: 475492.38
median_60: 381569.41
mean_4_dow5_2017: 190093.34
mean_3: 166122.57
no_promo_mean_140_decay: 96758.07
mean_3_decay: 85042.86
no_promo_mean_3_decay: 58562.36
median_3: 58532.04
mean_30: 57050.80
mean_30_decay: 52495.30
no_promo_mean_60_decay: 34808.68
item_day_2_2017: 26828.98
item_day_1_2017: 20069.69
item_min_3: 12698.42
no_promo_mean_3: 12273.45
day_2_2017: 9521.31
no_promo_mean_7_decay: 7637.29
no_promo_mean_30_decay: 4826.41
last_has

Step 14
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.579661	valid_1's l2: 0.553568
[100]	training's l2: 0.455849	valid_1's l2: 0.436223
[150]	training's l2: 0.41611	valid_1's l2: 0.399402
[200]	training's l2: 0.401837	valid_1's l2: 0.386452
Did not meet early stopping. Best iteration is:
[200]	training's l2: 0.401837	valid_1's l2: 0.386452
mean_140_decay: 5975247.40
mean_60_decay: 2189960.53
mean_20_dow6_2017: 1407427.60
promo_13: 673099.41
no_promo_mean_30: 513461.80
no_promo_mean_60: 486141.58
mean_4_dow6_2017: 456197.73
mean_30: 444483.30
median_60: 419047.49
no_promo_mean_140_decay: 147561.76
mean_3_decay: 117308.58
median_3: 81277.99
mean_60: 58682.60
no_promo_mean_60_decay: 52668.54
item_day_1_2017: 38503.38
day_1_2017: 28991.37
mean_30_decay: 22480.93
no_promo_mean_3_decay: 22145.53
no_promo_mean_7_decay: 10027.73
last_has_promo_day_in_after_15_days: 8775.72
mean_20_dow5_2017: 8708.45
no_promo_mean_30_decay: 7959.82
item_no_promo_mean_3_de

Step 15
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.580778	valid_1's l2: 0.553236
[100]	training's l2: 0.446382	valid_1's l2: 0.428408
[150]	training's l2: 0.404997	valid_1's l2: 0.390618
[200]	training's l2: 0.39041	valid_1's l2: 0.377782
Did not meet early stopping. Best iteration is:
[200]	training's l2: 0.39041	valid_1's l2: 0.377782
mean_140_decay: 4922602.20
mean_60_decay: 1639741.79
median_60: 1143991.96
mean_20_dow0_2017: 1090168.10
mean_30: 1032627.91
promo_14: 1021909.65
no_promo_mean_30: 977593.76
no_promo_mean_60: 530146.02
mean_4_dow0_2017: 437451.28
median_30: 277610.00
no_promo_mean_140_decay: 167587.80
median_7: 147675.47
no_promo_mean_60_decay: 97691.14
mean_7_decay: 81745.82
mean_7: 74687.24
mean_30_decay: 55452.67
no_promo_mean_7_decay: 33946.56
item_day_1_2017: 19712.07
has_promo_mean_30: 16648.20
no_promo_mean_3_decay: 15939.60
mean_20_dow2_2017: 10331.96
mean_60: 5005.99
no_promo_mean_30_decay: 4414.11
no_promo_mean_140: 25

Step 16
Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.546717	valid_1's l2: 0.541645
[100]	training's l2: 0.436361	valid_1's l2: 0.43717
[150]	training's l2: 0.40106	valid_1's l2: 0.404724
[200]	training's l2: 0.388304	valid_1's l2: 0.393096
Did not meet early stopping. Best iteration is:
[200]	training's l2: 0.388304	valid_1's l2: 0.393096
mean_140_decay: 3250599.96
mean_60_decay: 1558497.58
mean_30: 1455149.10
no_promo_mean_30: 1260559.97
median_60: 1077638.88
mean_20_dow1_2017: 925740.76
no_promo_mean_60: 658485.42
last_has_promo_day_in_after_15_days: 381012.13
promo_15: 234748.61
median_30: 225023.46
no_promo_mean_140_decay: 140745.24
mean_4_dow1_2017: 114912.91
mean_60: 99042.83
no_promo_mean_7: 67845.09
median_7: 43782.92
mean_7: 36648.69
mean_20_dow2_2017: 31244.95
no_promo_mean_60_decay: 30278.96
no_promo_mean_7_decay: 30006.51
mean_30_decay: 23950.00
item_day_1_2017: 20936.80
no_promo_mean_14: 16073.66
day_1_2017: 8810.75
no_promo_mean_30_

Validation mse: 0.39243124255652023
nwrmsle = 0.6266169722559898
Making submission...
